## Import packages

In [1]:
import argparse
import logging
import math
import os
import random
from pathlib import Path

import datasets
import numpy as np
import torch
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AdamW,
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
)
from utils_qa import postprocess_qa_predictions

logger = logging.getLogger(__name__)

## args

In [2]:
parser = argparse.ArgumentParser(
    description="Finetune a transformers model on a Question Answering task")

# data
parser.add_argument(
    "--dataset_name",
    type=str,
    default=None,
    help="The name of the dataset to use (via the datasets library).",
)
parser.add_argument(
    "--dataset_config_name",
    type=str,
    default=None,
    help="The configuration name of the dataset to use (via the datasets library).",
)

parser.add_argument("--do_train", action="store_true",
                    help="To do train on the question answering model")
parser.add_argument("--do_eval", action="store_true",
                    help="To do eval on the question answering model")
parser.add_argument("--do_predict", action="store_true",
                    help="To do prediction on the question answering model")
parser.add_argument(
    "--train_file", type=str, default="./data/QA_train.json", help="A csv or a json file containing the training data."
)
parser.add_argument(
    "--validation_file", type=str, default="./data/QA_valid.json", help="A csv or a json file containing the validation data."
)
parser.add_argument(
    "--test_file", type=str, default="./data/QA_test.json", help="A csv or a json file containing the Prediction data."
)
parser.add_argument("--output_dir", type=str, default="./qa",
                    help="Where to store the final model.")
parser.add_argument(
    "--output_csv", type=str, default="./data/test_submission.csv", help="A csv or a json file containing the Prediction data."
)
parser.add_argument(
    "--max_seq_length",
    type=int,
    default=384,
    help="The maximum total input sequence length after tokenization. Sequences longer than this will be truncated,"
    " sequences shorter will be padded if `--pad_to_max_lengh` is passed.",
)
parser.add_argument(
    "--pad_to_max_length",
    action="store_true",
    help="If passed, pad all samples to `max_seq_length`. Otherwise, dynamic padding is used.",
)
parser.add_argument(
    "--preprocessing_num_workers", type=int, default=4, help="preprocessing_num_workers"
)

# model
parser.add_argument(
    "--model_name_or_path",
    type=str,
    default="bert-base-chinese",
    help="Path to pretrained model or model identifier from huggingface.co/models.",
    # required=True,
)
parser.add_argument(
    "--config_name",
    type=str,
    default=None,
    help="Pretrained config name or path if not the same as model_name",
)
parser.add_argument(
    "--tokenizer_name",
    type=str,
    default=None,
    help="Pretrained tokenizer name or path if not the same as model_name",
)
parser.add_argument(
    "--use_slow_tokenizer",
    action="store_true",
    help="If passed, will use a slow tokenizer (not backed by the 🤗 Tokenizers library).",
)
parser.add_argument(
    "--per_device_train_batch_size",
    type=int,
    default=4,
    help="Batch size (per device) for the training dataloader.",
)
parser.add_argument(
    "--per_device_eval_batch_size",
    type=int,
    default=4,
    help="Batch size (per device) for the evaluation dataloader.",
)
parser.add_argument(
    "--learning_rate",
    type=float,
    default=3e-5,
    help="Initial learning rate (after the potential warmup period) to use.",
)
parser.add_argument("--weight_decay", type=float,
                    default=0.0, help="Weight decay to use.")
parser.add_argument("--num_train_epochs", type=int, default=3,
                    help="Total number of training epochs to perform.")
parser.add_argument(
    "--max_train_steps",
    type=int,
    default=None,
    help="Total number of training steps to perform. If provided, overrides num_train_epochs.",
)
parser.add_argument(
    "--gradient_accumulation_steps",
    type=int,
    default=8,
    help="Number of updates steps to accumulate before performing a backward/update pass.",
)
parser.add_argument(
    "--lr_scheduler_type",
    type=SchedulerType,
    default="linear",
    help="The scheduler type to use.",
    choices=["linear", "cosine", "cosine_with_restarts",
             "polynomial", "constant", "constant_with_warmup"],
)
parser.add_argument(
    "--num_warmup_steps", type=int, default=0, help="Number of steps for the warmup in the lr scheduler."
)
parser.add_argument("--seed", type=int, default=42,
                    help="A seed for reproducible training.")
parser.add_argument(
    "--doc_stride",
    type=int,
    default=128,
    help="When splitting up a long document into chunks how much stride to take between chunks.",
)
parser.add_argument(
    "--n_best_size",
    type=int,
    default=20,
    help="The total number of n-best predictions to generate when looking for an answer.",
)
parser.add_argument(
    "--null_score_diff_threshold",
    type=float,
    default=0.0,
    help="The threshold used to select the null answer: if the best answer has a score that is less than "
    "the score of the null answer minus this threshold, the null answer is selected for this example. "
    "Only useful when `version_2_with_negative=True`.",
)
parser.add_argument(
    "--version_2_with_negative",
    type=bool,
    default=False,
    help="If true, some of the examples do not have an answer.",
)
parser.add_argument(
    "--max_answer_length",
    type=int,
    default=30,
    help="The maximum length of an answer that can be generated. This is needed because the start "
    "and end predictions are not conditioned on one another.",
)
parser.add_argument(
    "--max_train_samples",
    type=int,
    default=None,
    help="For debugging purposes or quicker training, truncate the number of training examples to this "
    "value if set.",
)
parser.add_argument(
    "--max_eval_samples",
    type=int,
    default=None,
    help="For debugging purposes or quicker training, truncate the number of evaluation examples to this "
    "value if set.",
)
parser.add_argument(
    "--max_predict_samples",
    type=int,
    default=None,
    help="For debugging purposes or quicker training, truncate the number of prediction examples to this",
)
parser.add_argument(
    "--overwrite_cache", type=bool, default=False, help="Overwrite the cached training and evaluation sets"
)

# train
args = parser.parse_args(
    args=[
        "--pad_to_max_length",
        "--max_train_samples", '100',
        "--max_eval_samples", '100',
        "--do_train",
        "--do_eval"
    ]
)

# test
# args = parser.parse_args(
#     args=[
#     "--pad_to_max_length",
#     "--model_name_or_path", './qa',
#     "--max_predict_samples", '100',
#     "--do_predict"]
# )

# Sanity checks
if (
    args.dataset_name is None
    and args.train_file is None
    and args.validation_file is None
    and args.test_file is None
):
    raise ValueError(
        "Need either a dataset name or a training/validation/test file.")
else:
    if args.train_file is not None:
        extension = args.train_file.split(".")[-1]
        assert extension in [
            "csv", "json"], "`train_file` should be a csv or a json file."
    if args.validation_file is not None:
        extension = args.validation_file.split(".")[-1]
        assert extension in [
            "csv", "json"], "`validation_file` should be a csv or a json file."
    if args.test_file is not None:
        extension = args.test_file.split(".")[-1]
        assert extension in [
            "csv", "json"], "`test_file` should be a csv or a json file."


print(args)


Namespace(config_name=None, dataset_config_name=None, dataset_name=None, do_eval=True, do_predict=False, do_train=True, doc_stride=128, gradient_accumulation_steps=8, learning_rate=3e-05, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, max_answer_length=30, max_eval_samples=100, max_predict_samples=None, max_seq_length=384, max_train_samples=100, max_train_steps=None, model_name_or_path='bert-base-chinese', n_best_size=20, null_score_diff_threshold=0.0, num_train_epochs=3, num_warmup_steps=0, output_csv='./data/test_submission.csv', output_dir='./qa', overwrite_cache=False, pad_to_max_length=True, per_device_eval_batch_size=4, per_device_train_batch_size=4, preprocessing_num_workers=4, seed=42, test_file='./data/QA_test.json', tokenizer_name=None, train_file='./data/QA_train.json', use_slow_tokenizer=False, validation_file='./data/QA_valid.json', version_2_with_negative=False, weight_decay=0.0)


## Initialize accelerator, logger

In [3]:
# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()
# Make one log on every process with the configuration for debugging.
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state)

# Setup logging, we only want one process per machine to log things on the screen.
# accelerator.is_local_main_process is only True for one process per machine.
logger.setLevel(logging.INFO if accelerator.is_local_main_process else logging.ERROR)
if accelerator.is_local_main_process:
    datasets.utils.logging.set_verbosity_warning()
    transformers.utils.logging.set_verbosity_info()
else:
    datasets.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_error()

# If passed along, set the training seed now.
if args.seed is not None:
    set_seed(args.seed)

# Handle the repository creation
if accelerator.is_main_process:
    if args.output_dir is not None:
        os.makedirs(args.output_dir, exist_ok=True)
accelerator.wait_for_everyone()


04/01/2022 01:26:56 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda
Mixed precision type: no



## Dataset

In [4]:
# Get the datasets: you can either provide your own CSV/JSON/TXT training and evaluation files (see below)
# or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
# (the dataset will be downloaded automatically from the datasets Hub).
#
# For CSV/JSON files, this script will use the column called 'text' or the first column if no column called
# 'text' is found. You can easily tweak this behavior (see below).
#
# In distributed training, the load_dataset function guarantee that only one local process can concurrently
# download the dataset.
if args.dataset_name is not None:
    # Downloading and loading a dataset from the hub.
    raw_datasets = load_dataset(args.dataset_name, args.dataset_config_name)
else:
    data_files = {}
    if args.train_file is not None and args.do_train:
        data_files["train"] = args.train_file
        extension = args.train_file.split(".")[-1]
    if args.validation_file is not None and args.do_eval:
        data_files["validation"] = args.validation_file
        extension = args.validation_file.split(".")[-1]
    if args.test_file is not None and args.do_predict:
        data_files["test"] = args.test_file
        extension = args.test_file.split(".")[-1]
    
    raw_datasets = load_dataset(extension, data_files=data_files, field="data")

04/01/2022 01:26:57 - WARNING - datasets.builder - Using custom data configuration default-8c4fdf57250df383
04/01/2022 01:26:57 - WARNING - datasets.builder - Reusing dataset json (/home/bosh/.cache/huggingface/datasets/json/default-8c4fdf57250df383/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)
100%|██████████| 2/2 [00:00<00:00, 174.77it/s]


In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'paragraphs', 'relevant', 'answers', 'context'],
        num_rows: 21714
    })
    validation: Dataset({
        features: ['id', 'question', 'paragraphs', 'relevant', 'answers', 'context'],
        num_rows: 3009
    })
})

## Load pretrained model and tokenizer

In [6]:
# Load pretrained model and tokenizer
#
# In distributed training, the .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.

config = AutoConfig.from_pretrained(args.model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=True)
model = AutoModelForQuestionAnswering.from_pretrained(
            args.model_name_or_path,
            from_tf=bool(".ckpt" in args.model_name_or_path),
            config=config,
        )

loading configuration file https://huggingface.co/bert-base-chinese/resolve/main/config.json from cache at /home/bosh/.cache/huggingface/transformers/6cc404ca8136bc87bae0fb24f2259904943d776a6c5ddc26598bbdc319476f42.0f9bcd8314d841c06633e7b92b04509f1802c16796ee67b0f1177065739e24ae
Model config BertConfig {
  "_name_or_path": "bert-base-chinese",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transf

## Preprocessing the datasets.

In [7]:
# Preprocessing the datasets.
# Preprocessing is slighlty different for training and evaluation.

question_column_name = "question" 
context_column_name = "context" 
answer_column_name = "answers" 

# Padding side determines if we do (question|context) or (context|question).
pad_on_right = tokenizer.padding_side == "right"

if args.max_seq_length > tokenizer.model_max_length:
    logger.warning(
        f"The max_seq_length passed ({args.max_seq_length}) is larger than the maximum length for the"
        f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}."
    )

max_seq_length = min(args.max_seq_length, tokenizer.model_max_length)

## train preprocessing

In [8]:
# Training preprocessing
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples[question_column_name] = [q.lstrip() for q in examples[question_column_name]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples[question_column_name if pad_on_right else context_column_name],
        examples[context_column_name if pad_on_right else question_column_name],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_seq_length,
        stride=args.doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length" if args.pad_to_max_length else False,
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")  # ex: [0,1,2,2,3,4,4,4...]
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")  # ex: [off_0, off_1, off_2, off_2_1, off_3, off_4, off_4_1, off_4_2...]

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    #  若有些 QA context 太常會被分成好幾段，offset_mapping長度就是所有分段 QA input 的數量 
    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        # None: 為特殊字符, 0: 該 token 屬於 question, 1: 該 token 屬於 context
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        # 第幾個 QA pair （一個 QA pair可以分成很多段，所以可能多段對應到同一個 QA pair）
        sample_index = sample_mapping[i]
        answers = examples[answer_column_name][sample_index]
        # If no answers are given, set the cls_index as answer.
        # 此 QA 沒有給 answer就把 token start_positions, token end_positions 都定在cls的位置
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            # 找出該 span 第一個 context token 的位置
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            # 找出該 span 最後一個 context token 的位置
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            # 若答案不在一個過長的 QA 的其中一段，就把這段 token start_positions, token end_positions 都定在cls的位置
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # 確認 answer 一定在這段 span 的 context 中
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)  # 找出 answer start token 在該 span 的位置
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)  # 找出 answer end token 在該 span 的位置

    return tokenized_examples

## create train dataset

In [9]:
if args.do_train:
    if "train" not in raw_datasets:
        raise ValueError("--do_train requires a train dataset")
    raw_train_dataset = raw_datasets["train"]

    # select train data
    if args.max_train_samples is not None:
        # We will select sample from whole data if agument is specified
        raw_train_dataset = raw_train_dataset.select(range(args.max_train_samples))
    print("\n\n" + "="*100 + "\n\n")
    print(raw_train_dataset)

    # Create train feature from dataset
    with accelerator.main_process_first():
        train_dataset = raw_train_dataset.map(
            prepare_train_features,
            batched=True,
            num_proc=args.preprocessing_num_workers,
            remove_columns=raw_datasets["train"].column_names,
            load_from_cache_file=not args.overwrite_cache,
            desc="Running tokenizer on train dataset",
        )
    print("\n\n" + "="*100 + "\n\n")
    print(train_dataset)  # 一個 QA pair 可能會被切好幾段，所以數量可能會大於 raw_train_dataset
        

04/01/2022 01:27:07 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /home/bosh/.cache/huggingface/datasets/json/default-8c4fdf57250df383/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-72a8a72add1db864.arrow
04/01/2022 01:27:07 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /home/bosh/.cache/huggingface/datasets/json/default-8c4fdf57250df383/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-7f4c66b2f298ad9e.arrow
04/01/2022 01:27:07 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /home/bosh/.cache/huggingface/datasets/json/default-8c4fdf57250df383/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-4e0c1715939c4de7.arrow
04/01/2022 01:27:07 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /home/bosh/.cache/huggingface/datasets/json/default-8c4fdf57250df383/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6





Dataset({
    features: ['id', 'question', 'paragraphs', 'relevant', 'answers', 'context'],
    num_rows: 100
})




Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 173
})


## Validation preprocessing

In [10]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples[question_column_name] = [q.lstrip() for q in examples[question_column_name]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples[question_column_name if pad_on_right else context_column_name],
        examples[context_column_name if pad_on_right else question_column_name],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_seq_length,
        stride=args.doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length" if args.pad_to_max_length else False,
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")  # ex: [0,1,2,2,3,4,4,4...]

    # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
    # corresponding example_id and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        # 原本 question tokens 和 特殊字符 tokens 也有 offset pair (startIdx, endIdx), 將除了 context 之外的 offset mapping 都轉成 None 就能快速分辨哪些是 context
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

## create validation dataset

In [11]:
if args.do_eval:
    if "validation" not in raw_datasets:
        raise ValueError("--do_eval requires a validation dataset")
    raw_eval_dataset = raw_datasets["validation"]
    if args.max_eval_samples is not None:
        # We will select sample from whole data
        raw_eval_dataset = raw_eval_dataset.select(range(args.max_eval_samples))
    print("\n\n" + "="*100 + "\n\n")
    print(raw_eval_dataset)
    # Validation Feature Creation
    with accelerator.main_process_first():
        eval_dataset = raw_eval_dataset.map(
            prepare_validation_features,
            batched=True,
            num_proc=args.preprocessing_num_workers,
            remove_columns=raw_datasets["validation"].column_names,
            load_from_cache_file=not args.overwrite_cache,
            desc="Running tokenizer on validation dataset",
        )
    print("\n\n" + "="*100 + "\n\n")
    #  沒有 'start_positions', 'end_positions' 但有 'offset_mapping', 'example_id'
    print(eval_dataset)  # 一個 QA pair 可能會被切好幾段，所以數量可能會大於 raw_eval_dataset






Dataset({
    features: ['id', 'question', 'paragraphs', 'relevant', 'answers', 'context'],
    num_rows: 100
})


04/01/2022 01:27:08 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /home/bosh/.cache/huggingface/datasets/json/default-8c4fdf57250df383/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-ed2b6e6797df23ac.arrow
04/01/2022 01:27:08 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /home/bosh/.cache/huggingface/datasets/json/default-8c4fdf57250df383/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-5849039a9a303d88.arrow
04/01/2022 01:27:08 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /home/bosh/.cache/huggingface/datasets/json/default-8c4fdf57250df383/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-d498db1a48db73ff.arrow
04/01/2022 01:27:08 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /home/bosh/.cache/huggingface/datasets/json/default-8c4fdf57250df383/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6





Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 184
})


## create predict dataset

In [12]:
if args.do_predict:
    if "test" not in raw_datasets:
        raise ValueError("--do_predict requires a test dataset")
    raw_predict_dataset = raw_datasets["test"]
    if args.max_predict_samples is not None:
        # We will select sample from whole data
        raw_predict_dataset = raw_predict_dataset.select(range(args.max_predict_samples))
    print("\n\n" + "="*100 + "\n\n")
    print(raw_predict_dataset)
    # Predict Feature Creation
    with accelerator.main_process_first():
        predict_dataset = raw_predict_dataset.map(
            prepare_validation_features,
            batched=True,
            num_proc=args.preprocessing_num_workers,
            remove_columns=raw_datasets["test"].column_names,
            load_from_cache_file=not args.overwrite_cache,
            desc="Running tokenizer on prediction dataset",
        )
    print("\n\n" + "="*100 + "\n\n")
    #  沒有 'start_positions', 'end_positions' 但有 'offset_mapping', 'example_id'
    print(predict_dataset)  # 一個 QA pair 可能會被切好幾段，所以數量可能會大於 raw_eval_dataset


## DataLoaders creation

In [13]:
if args.pad_to_max_length:
    # If padding was already done ot max length, we use the default data collator that will just convert everything
    # to tensors.
    data_collator = default_data_collator
else:
    # Otherwise, `DataCollatorWithPadding` will apply dynamic padding for us (by padding to the maximum length of
    # the samples passed). When using mixed precision, we add `pad_to_multiple_of=8` to pad all tensors to multiple
    # of 8s, which will enable the use of Tensor Cores on NVIDIA hardware with compute capability >= 7.5 (Volta).
    data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=(8 if accelerator.use_fp16 else None))

if args.do_train and args.do_eval:
    train_dataloader = DataLoader(
        train_dataset, shuffle=True, collate_fn=data_collator, batch_size=args.per_device_train_batch_size
    )

    eval_dataset_for_model = eval_dataset.remove_columns(["example_id", "offset_mapping"])
    eval_dataloader = DataLoader(
        eval_dataset_for_model, shuffle=False, collate_fn=data_collator, batch_size=args.per_device_eval_batch_size
    )
    metric = load_metric("squad_v2" if args.version_2_with_negative else "squad")

if args.do_predict:
    predict_dataset_for_model = predict_dataset.remove_columns(["example_id", "offset_mapping"])
    predict_dataloader = DataLoader(
        predict_dataset_for_model, shuffle=False, collate_fn=data_collator, batch_size=args.per_device_eval_batch_size
    )

## Post-processing:

In [14]:
def post_processing_function(examples, features, predictions, stage="eval"):
    # Post-processing: we match the start logits and end logits to answers in the original context.
    predictions = postprocess_qa_predictions(
        examples=examples,
        features=features,
        predictions=predictions,
        version_2_with_negative=args.version_2_with_negative,
        n_best_size=args.n_best_size,
        max_answer_length=args.max_answer_length,
        null_score_diff_threshold=args.null_score_diff_threshold,
        output_dir=args.output_dir,
        prefix=stage,
    )
    # Format the result to the format the metric expects.
    if args.version_2_with_negative:
        formatted_predictions = [
            {"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()
        ]
    else:
        formatted_predictions = [{"id": k, "prediction_text": v} for k, v in predictions.items()]

    if stage == "eval":
        references = [{"id": ex["id"], "answers": ex[answer_column_name]} for ex in examples]
        return EvalPrediction(predictions=formatted_predictions, label_ids=references)
    elif stage == "predict":
        return formatted_predictions



In [15]:
# Create and fill numpy array of size len_of_validation_data * max_length_of_output_tensor
def create_and_fill_np_array(start_or_end_logits, dataset, max_len):
    """
    Create and fill numpy array of size len_of_validation_data * max_length_of_output_tensor
    Args:
        start_or_end_logits(:obj:`tensor`):
            This is the output predictions of the model. We can only enter either start or end logits.
        eval_dataset: Evaluation dataset
        max_len(:obj:`int`):
            The maximum length of the output tensor. ( See the model.eval() part for more details )
    """

    step = 0
    # create a numpy array and fill it with -100.
    logits_concat = np.full((len(dataset), max_len), -100, dtype=np.float64)
    # Now since we have create an array now we will populate it with the outputs gathered using accelerator.gather
    for i, output_logit in enumerate(start_or_end_logits):  # populate columns
        # We have to fill it such that we have to take the whole tensor and replace it on the newly created array
        # And after every iteration we have to change the step

        batch_size = output_logit.shape[0]
        cols = output_logit.shape[1]

        if step + batch_size < len(dataset):
            logits_concat[step : step + batch_size, :cols] = output_logit
        else:
            logits_concat[step:, :cols] = output_logit[: len(dataset) - step]

        step += batch_size

    return logits_concat

## prepare hyperparameters

In [16]:
# Optimizer
# Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

if args.do_train and args.do_eval:
    # Prepare everything with our `accelerator`.
    model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader
    )

if args.do_predict:
    model, optimizer, predict_dataloader = accelerator.prepare(
    model, optimizer, predict_dataloader
)

# Note -> the training dataloader needs to be prepared before we grab his length below (cause its length will be
# shorter in multiprocess)

/home/bosh/anaconda3/envs/adl/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [17]:
if args.do_train:    
    # Scheduler and math around the number of training steps.
    num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
    if args.max_train_steps is None:
        args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
    else:
        args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

    lr_scheduler = get_scheduler(
        name=args.lr_scheduler_type,
        optimizer=optimizer,
        num_warmup_steps=args.num_warmup_steps,
        num_training_steps=args.max_train_steps,
    )


## Train

In [18]:
if args.do_train and args.do_eval:
    # Train!
    total_batch_size = args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

    logger.info("***** Running training *****")
    logger.info(f"  Num examples = {len(train_dataset)}")
    logger.info(f"  Num Epochs = {args.num_train_epochs}")
    logger.info(f"  Instantaneous batch size per device = {args.per_device_train_batch_size}")
    logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
    logger.info(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
    logger.info(f"  Total optimization steps = {args.max_train_steps}")
    print("\n\n" + "="*100 + "\n\n")

    # Only show the progress bar once on each machine.
    progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)
    completed_steps = 0

    eval_EM = []
    for epoch in range(args.num_train_epochs):
        model.train()
        for step, batch in enumerate(train_dataloader):
            outputs = model(**batch)
            loss = outputs.loss
            loss = loss / args.gradient_accumulation_steps
            accelerator.backward(loss)
            if step % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                progress_bar.update(1)
                completed_steps += 1

            if completed_steps >= args.max_train_steps:
                break


        # Evaluation
        print("\n\n" + "=" * 100 + "\n\n")
        logger.info("***** Running Evaluation *****")
        logger.info(f"  Num examples = {len(eval_dataset)}")
        logger.info(f"  Batch size = {args.per_device_eval_batch_size}")

        all_start_logits = []
        all_end_logits = []
        for step, batch in enumerate(eval_dataloader):
            model.eval()
            with torch.no_grad():
                outputs = model(**batch)
                start_logits = outputs.start_logits
                end_logits = outputs.end_logits

                if not args.pad_to_max_length:  # necessary to pad predictions and labels for being gathered
                    start_logits = accelerator.pad_across_processes(start_logits, dim=1, pad_index=-100)
                    end_logits = accelerator.pad_across_processes(end_logits, dim=1, pad_index=-100)

                all_start_logits.append(accelerator.gather(start_logits).cpu().numpy())
                all_end_logits.append(accelerator.gather(end_logits).cpu().numpy())

        max_len = max([x.shape[1] for x in all_start_logits])  # Get the max_length of the tensor

        # concatenate the numpy array
        start_logits_concat = create_and_fill_np_array(all_start_logits, eval_dataset, max_len)
        end_logits_concat = create_and_fill_np_array(all_end_logits, eval_dataset, max_len)

        # delete the list of numpy arrays
        del all_start_logits
        del all_end_logits

        outputs_numpy = (start_logits_concat, end_logits_concat)
        prediction = post_processing_function(raw_eval_dataset, eval_dataset, outputs_numpy)
        eval_metric = metric.compute(predictions=prediction.predictions, references=prediction.label_ids)
        logger.info(f"Evaluation metrics: {eval_metric}")
        eval_EM.append(eval_metric["exact_match"])

        # save check point
        if args.output_dir and epoch < args.num_train_epochs - 1:
            accelerator.wait_for_everyone()
            unwrapped_model = accelerator.unwrap_model(model)
            unwrapped_model.save_pretrained(args.output_dir, save_function=accelerator.save)

    # save model
    if args.output_dir is not None:
        np.save(f'{args.output_dir}/eval_EM', np.array(eval_EM))
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(args.output_dir, save_function=accelerator.save)
        if accelerator.is_main_process:
            tokenizer.save_pretrained(args.output_dir)



04/01/2022 01:27:12 - INFO - __main__ - ***** Running training *****
04/01/2022 01:27:12 - INFO - __main__ -   Num examples = 173
04/01/2022 01:27:12 - INFO - __main__ -   Num Epochs = 3
04/01/2022 01:27:12 - INFO - __main__ -   Instantaneous batch size per device = 4
04/01/2022 01:27:12 - INFO - __main__ -   Total train batch size (w. parallel, distributed & accumulation) = 32
04/01/2022 01:27:12 - INFO - __main__ -   Gradient Accumulation steps = 8
04/01/2022 01:27:12 - INFO - __main__ -   Total optimization steps = 18


 39%|███▉      | 7/18 [00:03<00:03,  2.80it/s]04/01/2022 01:27:15 - INFO - __main__ - ***** Running Evaluation *****
04/01/2022 01:27:15 - INFO - __main__ -   Num examples = 184
04/01/2022 01:27:15 - INFO - __main__ -   Batch size = 4
100%|██████████| 100/100 [00:00<00:00, 283.74it/s]
04/01/2022 01:27:16 - INFO - __main__ - Evaluation metrics: {'exact_match': 7.0, 'f1': 7.0}
Configuration saved in ./qa/config.json
Model weights saved in ./qa/pytorch_model.bin
 78%|███████▊  | 14/18 [00:08<00:01,  2.31it/s]04/01/2022 01:27:20 - INFO - __main__ - ***** Running Evaluation *****
04/01/2022 01:27:20 - INFO - __main__ -   Num examples = 184
04/01/2022 01:27:20 - INFO - __main__ -   Batch size = 4
100%|██████████| 100/100 [00:00<00:00, 281.47it/s]
04/01/2022 01:27:22 - INFO - __main__ - Evaluation metrics: {'exact_match': 10.0, 'f1': 10.0}
Configuration saved in ./qa/config.json
Model weights saved in ./qa/pytorch_model.bin
100%|██████████| 18/18 [00:12<00:00,  1.46it/s]04/01/2022 01:27:24 - 

## Predict

In [19]:
# Prediction
if args.do_predict:
    logger.info("***** Running Prediction *****")
    logger.info(f"  Num examples = {len(predict_dataset)}")
    logger.info(f"  Batch size = {args.per_device_eval_batch_size}")

    all_start_logits = []
    all_end_logits = []
    for step, batch in enumerate(predict_dataloader):
        model.eval()
        with torch.no_grad():
            outputs = model(**batch)
            start_logits = outputs.start_logits
            end_logits = outputs.end_logits

            if not args.pad_to_max_length:  # necessary to pad predictions and labels for being gathered
                start_logits = accelerator.pad_across_processes(start_logits, dim=1, pad_index=-100)
                end_logits = accelerator.pad_across_processes(start_logits, dim=1, pad_index=-100)

            all_start_logits.append(accelerator.gather(start_logits).cpu().numpy())
            all_end_logits.append(accelerator.gather(end_logits).cpu().numpy())

    max_len = max([x.shape[1] for x in all_start_logits])  # Get the max_length of the tensor
    # concatenate the numpy array
    start_logits_concat = create_and_fill_np_array(all_start_logits, predict_dataset, max_len)
    end_logits_concat = create_and_fill_np_array(all_end_logits, predict_dataset, max_len)

    # delete the list of numpy arrays
    del all_start_logits
    del all_end_logits

    outputs_numpy = (start_logits_concat, end_logits_concat)

    # [{"id": k, "prediction_text": v} for k, v in predictions.items()]
    predictions = post_processing_function(raw_predict_dataset, predict_dataset, outputs_numpy, stage="predict")

    # write csv file
    with open(args.output_csv, 'w') as f:
            f.write('id,answer\n')
            for i, prediction in enumerate(predictions):
                f.write(f'{prediction["id"]},{prediction["prediction_text"]}\n')
